# Mounting Drive

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [5]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

#print 'Files in Drive:'
!ls drive/

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

5th Sem  Advanced Java CPD  Colab Notebooks  created.txt  Mom School Stuff
6th Sem  ChatBot	    ContentBlocker   EYRC


In [6]:
!pwd
!ls

/content
datalab  drive


# Uploading Text File

In [33]:
from google.colab import files
src = list(files.upload().values())[0]
open('test.docx','wb').write(src)

Saving test.docx to test (2).docx


41849

In [77]:
from google.colab import files
src = list(files.upload().values())[0]
open('vgg16.py','wb').write(src)
import vgg16

Saving vgg16.py to vgg16.py


# Importing Pre Trained Model

In [0]:
import numpy as np
import os
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image

In [79]:
PATH = os.getcwd()
print(PATH)
data_path = PATH + '/drive/ContentBlocker/Dataset'
print(data_path)

/content
/content/drive/ContentBlocker/Dataset


In [80]:

image_input = Input(shape=(224, 224, 3))

model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

#model.summary()

553467904/553467096 [==============================] - 12s 0us/step


In [0]:

last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(3, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)

In [0]:
weights_path = data_path + "/model_weights.h5"

In [0]:
custom_vgg_model2.load_weights(weights_path)

# Applying model on images of docs

In [0]:
#image_path = data_path + '/Train/Close_to_abusive/1.jpg'

In [117]:
from google.colab import files
src = list(files.upload().values())[0]
open('test.docx','wb').write(src)

Saving test later.docx to test later (4).docx


277101

In [0]:
import zipfile

zipfname = 'test later (3).docx'
z = zipfile.ZipFile(zipfname)

In [111]:
!ls
!pwd

1.jpeg	     image4.jpeg  test (1).docx  test later (1).docx
datalab      image5.jpeg  test (2).docx  test later (2).docx
drive	     image6.jpeg  test (3).docx  test later (3).docx
image1.jpeg  ntest.docx   test (4).docx  test later.docx
image2.jpeg  __pycache__  test (5).docx  vgg16.py
image3.jpeg  read.jpeg	  test.docx
/content


In [119]:
#print list of valid attributes for ZipFile object
print(dir(z))

#print all files in zip archive
all_files = z.namelist()
print(all_files)

['NameToInfo', '_RealGetContents', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_allowZip64', '_comment', '_didModify', '_extract_member', '_filePassed', '_fileRefCnt', '_fpclose', '_lock', '_open_to_write', '_sanitize_windows_name', '_seekable', '_windows_illegal_name_trans_table', '_write_end_record', '_writecheck', '_writing', 'close', 'comment', 'compression', 'debug', 'extract', 'extractall', 'filelist', 'filename', 'fp', 'getinfo', 'infolist', 'mode', 'namelist', 'open', 'printdir', 'pwd', 'read', 'setpassword', 'start_dir', 'testzip', 'write', 'writestr']
['[Content_Types].xml', '_rels/.rels', 'word/_rels/document.xml.rels', 'word/document.xml', 'wo

In [120]:
images = [img_name for img_name in all_files if img_name.split('.')[-1] in ['jpg','jpeg','png']]

print(images)

['word/media/image5.jpeg', 'word/media/image4.jpeg', 'word/media/image3.jpeg', 'word/media/image2.jpeg', 'word/media/image1.jpeg', 'word/media/image6.jpeg']


In [19]:
#get all files in word/media/ directory
#images = list(filter(lambda x: x.startswith('word/media/'), all_files))
#print(images)
#print(len(images))

['word/media/image5.jpeg', 'word/media/image4.jpeg', 'word/media/image3.jpeg', 'word/media/image2.jpeg', 'word/media/image1.jpeg', 'word/media/image6.jpeg']


In [0]:
import os
from IPython.display import Image, display
from keras.preprocessing.image import load_img
from zipfile import ZipFile
from PIL import Image
from io import BytesIO
import PIL
import paradocx
from paradocx import Document
from paradocx.image import ImagePart
import cv2
import os
import tempfile
import zipfile
import shutil

In [0]:
!pip install paradocx

In [121]:
archive = ZipFile(zipfname, 'r')
doc = Document.from_file(zipfname)
filenames = [ig.split('/')[-1] for ig in images]
print(filenames)
done = []

['image5.jpeg', 'image4.jpeg', 'image3.jpeg', 'image2.jpeg', 'image1.jpeg', 'image6.jpeg']


In [122]:
#open an image and save it
for image_name in images:
  '''
  image1 = z.open(image_name).read()
  f = open(temp_image_name,'wb')
  f.write(image1)
  
  #Extract file
  z.extract(image_name, '/content')
  
  # load an image from file
  #image = load_img(extract_path + '/' + image_name.split('/')[-1], target_size=(224, 224))
  image = load_img('/content/'+temp_image_name, target_size=(224, 224))
  
  #print(type(image1))
  '''
  image_data = archive.read(image_name)
  img = cv2.imdecode(np.frombuffer(image_data, np.uint8), 1) 
  image_extension = image_name.split('.')[-1]
  cv2.imwrite(image_name.split('/')[-1],img)
  
  # load an image from file
  image = load_img(image_name.split('/')[-1], target_size=(224, 224))
  #image = load_img('/content/'+'1.'+image_extension, target_size=(224, 224))
  
  
  from keras.preprocessing.image import img_to_array
  # convert the image pixels to a numpy array
  image = img_to_array(image)

  # reshape data for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

  from keras.applications.vgg16 import preprocess_input
  # prepare the image for the VGG model
  image = preprocess_input(image)

  # predict the probability across all output classes
  yhat = custom_vgg_model2.predict(image)
  
  
  
  
  '''
  image1 = ImagePart(doc, '/'+image_name)
  image1.data = gausBlur
  print('start')
  '''

  print(yhat[0])
  
  #BLUR NON-ABUSIVE IMAGE
  if (yhat[0][0]<=0.5):
    print(image_name.split('/')[-1] + ' is Non-Abusive')
    gausBlur = cv2.GaussianBlur(np.asarray(img), (45,45),0) 
    for i in range(4):
      gausBlur = cv2.GaussianBlur(gausBlur, (45,45),0)  
    cv2.imwrite(image_name.split('/')[-1],gausBlur)

    tempdir = tempfile.mkdtemp()
    try:
      tempname = os.path.join(tempdir, zipfname)
      with zipfile.ZipFile(zipfname, 'r') as zipread:
          with zipfile.ZipFile(tempname, 'w') as zipwrite:
              for item in zipread.infolist():
                  print(item.filename)
                  if item.filename.split('/')[-1] != image_name.split('/')[-1]:
                      data = zipread.read(item.filename)
                      print('type in zipread:'+str(type(data))+'\ndata:'+str(data))
                      zipwrite.writestr(item, data)
                  else:
                    with open(image_name.split('/')[-1], "rb") as imageFile:
                      f = imageFile.read()
                      data = bytearray(f)
                      print('type in cv2:'+str(type(data))+'\ndata:'+str(data))
                    zipwrite.writestr(item, data)
                    done.append(image_name.split('/')[-1])
      shutil.move(tempname, zipfname)
    finally:
        shutil.rmtree(tempdir)

In [0]:
from google.colab import files
files.download(zipfname)
#files.download('image1.jpeg')
#files.download('1.jpeg')

In [27]:
!ls

datalab  drive	image1.jpeg  ntest.docx  test.docx
